In [288]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes


In [289]:

try:
    conexion = psycopg2.connect(
        database="facua",  
        user="postgres",  
        password="admin",
        host="localhost"
        )
    print("Conexión realizada con éxito")
except OperationalError as e:
        # Manejo de errores comunes
    if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("Contraseña inválida.")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Error de conexión.")
    else:
        print(f"Ocurrió un error: {e}")
    
    

Conexión realizada con éxito


In [290]:
cursor = conexion.cursor()
   

In [291]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Supermercados (
    id_supermercado INT PRIMARY KEY,
    nombre VARCHAR(255) 
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Productos (
    id_producto SERIAL PRIMARY KEY,
    id_supermercado INT,
    categoria VARCHAR(255),
    producto VARCHAR(255) ,
    marca VARCHAR(255),
    Volumen VARCHAR(255),
    FOREIGN KEY (id_supermercado) REFERENCES Supermercados(id_supermercado)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Precios (
    id_precio SERIAL PRIMARY KEY,
    id_supermercado INT,
    fecha DATE ,
    precio DECIMAL(10, 2),
    porcentaje INT ,
    FOREIGN KEY (id_supermercado) REFERENCES Supermercados(id_supermercado)
);
''')


conexion.commit()



In [292]:
df_mercado=pd.read_csv('../datos/datos_supermercados_limpios.csv')

In [293]:
df_mercado.dtypes

Unnamed: 0.1         int64
Unnamed: 0           int64
fecha               object
Precio             float64
Supermercado        object
Categoria           object
producto            object
Porcentaje         float64
Marca               object
Volumen             object
id_supermercado      int64
dtype: object

In [294]:
lista_tuplas_mercados = []
for fila in df_mercado[['id_supermercado','Supermercado']].values:
    lista_tuplas_mercados.append(tuple(fila))

lista_tuplas_mercados[1]

(1, 'mercadona')

In [295]:

query_insercion_super = '''
    insert into Supermercados (id_supermercado,nombre)
    values (%s,%s)  ON CONFLICT (id_supermercado) DO NOTHING '''

cursor.executemany(query_insercion_super, lista_tuplas_mercados)
conexion.commit()

In [296]:
lista_tuplas_productos = []
for fila in df_mercado[['id_supermercado','Categoria', 'producto', 'Marca','Volumen']].values:
    lista_tuplas_productos.append(tuple(fila))

lista_tuplas_productos[1]

(1, 'aceite de girasol', 'aceite de girasol refinado ', nan, nan)

In [297]:
query_insercion_productos = '''
    insert into Productos (id_supermercado,categoria, producto, Marca,Volumen)
    values
    (%s,%s, %s, %s, %s)''' 

cursor.executemany(query_insercion_productos, lista_tuplas_productos)
 
conexion.commit()

In [298]:
lista_tuplas_precios = []
for fila in df_mercado[['id_supermercado','fecha', 'Precio', 'Porcentaje']].values:
    lista_tuplas_precios.append(tuple(fila))

lista_tuplas_precios[1]

(1, '13/07/2024', 1.45, 0.0)

In [299]:
query_insercion_precios = '''
    INSERT INTO Precios (id_supermercado,fecha,Precio, Porcentaje)
    values
    (%s,%s, %s, %s)''' 

cursor.executemany(query_insercion_precios, lista_tuplas_precios)
 
conexion.commit()